In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [10]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [11]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [12]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110680  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,2.426196e+06
2018-02-28,2.047150e+06
2018-03-31,2.349675e+06
2018-04-30,2.454614e+06
2018-05-31,2.544352e+06
2018-06-30,2.423019e+06
2018-07-31,2.768263e+06
2018-08-31,2.736730e+06
2018-09-30,2.225399e+06


In [13]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    2.426196e+06
 2018-02-28    2.047150e+06
 2018-03-31    2.349675e+06
 2018-04-30    2.454614e+06
 2018-05-31    2.544352e+06
 2018-06-30    2.423019e+06
 2018-07-31    2.768263e+06
 2018-08-31    2.736730e+06
 2018-09-30    2.225399e+06
 2018-10-31    2.386349e+06
 2018-11-30    2.240899e+06
 2018-12-31    2.164994e+06
 2019-01-31    2.323962e+06
 2019-02-28    2.108688e+06
 2019-03-31    2.328613e+06
 2019-04-30    2.370742e+06
 2019-05-31    2.511093e+06
 2019-06-30    2.430931e+06
 2019-07-31    2.746332e+06
 2019-08-31    2.793068e+06
 2019-09-30    2.222043e+06
 2019-10-31    2.472890e+06
 2019-11-30    2.334252e+06
 2019-12-31    2.322174e+06
 2020-01-31    2.381053e+06
 2020-02-29    1.910885e+06
 2020-03-31    1.968050e+06
 2020-04-30    1.940690e+06
 2020-05-31    2.256569e+06
 2020-06-30    2.292317e+06
 2020-07-31    2.593939e+06
 2020-08-31    2.779493e+06
 2020-09-30    2.171347e+06
 2020-10-31    2.440135e+06
 2020-11-30    2.365601e+06

# ARIMA

In [14]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [15]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -1.435888
p-value : 0.564955
Critical Values :
	1%: -3.661429
	5%: -2.960525
	10%: -2.619319


In [16]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.678222
p-value : 0.004430
Critical Values :
	1%: -3.669920
	5%: -2.964071
	10%: -2.621171


In [17]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1160.619, Time=0.08 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1155.444, Time=0.04 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1156.581, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1151.396, Time=0.07 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1153.496, Time=0.10 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1152.645, Time=0.27 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1153.935, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1155.918, Time=0.07 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1157.792, Time=0.08 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1153.252, Time=0.15 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1153.204, Time=0.25 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1155.937, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1157.910, Time=0.09 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1160.022, Time=0.09 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1155.188, Time=0.18 sec
 ARIMA(3,0

ARIMA(order=(0, 0, 3), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [18]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(0,0,3))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 3)   Log Likelihood                -570.630
Method:                       css-mle   S.D. of innovations         188975.795
Date:                Sat, 11 Sep 2021   AIC                           1151.261
Time:                        02:17:01   BIC                           1159.949
Sample:                             0   HQIC                          1154.445
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.353e+06   6.69e+04     35.175      0.000    2.22e+06    2.48e+06
ma.L1.y        0.5527      0.127      4.338      0.000       0.303       0.802
ma.L2.y        0.2582      0.175      1.474      0.1

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [19]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([2497412.2845476 , 2352633.09857107]), array([188975.79473969, 215922.75162487]), array([[2127026.53290798, 2867798.03618723],
       [1929432.28194355, 2775833.9151986 ]]))
